# Funcion para hacer el scraping

In [ ]:
# BLOQUE 1: Importaciones y configuración inicial

import requests # para hacer peticiones HTTP (descargar paginas web)
from bs4 import BeautifulSoup # para analizar el HTML descargado
import sqlite3 
import pandas as pd 
import time 
import re # para expresiones regulares (extraer numeros de texto)
from urllib.parse import urljoin, quote # urljoin: Construir URLs completas a partir de relativas 
import json # para trabajar con JSON 
from concurrent.futures import ThreadPoolExecutor, as_completed # para hacer peticiones en paralelo

# OPTIMIZACIÓN: Session reutiliza la conexión TCP entre peticiones
# en vez de abrir y cerrar una conexión nueva cada vez
session = requests.Session()

print("✅ Librerías importadas correctamente") 
print("🕷️ Comenzando la infiltración en Books To Scrape...")  


# BLOQUE 2: Funciones de Web Scraping 

def get_all_categories ():
    ''' OBTIENE TODAS LAS CATEGORIAS DEL SITIO'''
    url = "https://books.toscrape.com" 
    respuesta = session.get(url) # se descarga la web (usando session en vez de requests)
    soup = BeautifulSoup(respuesta.content, 'html.parser')  
    # convierte texto HTML ilegible, en un objeto inteligente que se pueda navegar 
    
    categories = []
    nav_list = soup.find('ul', class_='nav nav-list')
    if nav_list:
        category_links = nav_list.find_all ('a') [1:] # Saltar "Books"
        for link in category_links:
            category_name = link.text.strip()  # Extrae el texto del enlace y quita espacios 
            category_url = urljoin(url, link['href'])  # Convierte la URL relativa a absoluta 
            categories.append({ 
                'name' : category_name,
                'url' : category_url 
            })
            # URL relativa es una ruta parcial 
            # URL absoluta es la ruta completa 
            
            
    print (f"🎯 Encontradas {len(categories)} categorias")
    return categories 


def book_quantity (book_url): 
    ''' OBTIENE LA CANTIDAD EN STOCK DE UN LIBRO DESDE SU PAGINA DE DETALLES  '''
    try:
        soup_quantity = BeautifulSoup(session.get(book_url).content,'html.parser') # descarga y parsea la pagina individual del libro 
        quantity_text = soup_quantity.select_one('p.instock.availability').get_text(strip=True) 
        # Busca el <p> con clases "instock" y "availability", extrae su texto limpio
        match = re.search(r'\((\d+)\)', quantity_text) # busca un patron para extraer el numero entero 
        if match: 
            return int (match.group(1)) # devuelve la cantidad encontrada 
        else:
            return 0 # si no se encuentra la cantidad, devuelve 0 
    except Exception as e :
        print (f"❌ Erorr obteniendo cantidad para {book_url}:{e}")
        return 0 # en caso de error, devuelve 0 
    


def scrape_books_from_page(page_url): 
    ''' SCRAPE LIBROS DE UNA PAGINA ESPECIFICA  '''
    response = session.get(page_url) # Descarga la pagina (usando session)
    soup = BeautifulSoup(response.content,'html.parser' ) # Parsea el HTML 
    
    books = []
    book_containers = soup.find_all ('article', class_ ='product_pod') 
    # cada libro esta dentro de un <article class="product_pod"> 
    
    for book in book_containers: 
        try:
            # TITULO
            title_element = book.find('h3').find('a') # busca la a dentro del h3 
            title = title_element['title'] # el titulo completo esta en el atributo "title"

            # URL DEL LIBRO PARA MAS DETALLES 
            book_url = urljoin(page_url, title_element['href']) 
            # cosntruye la url absoluta del libro a partir de su href relativo
            
            
            # PRECIO
            price_element = book.find('p', class_= 'price_color') 
            price_text = price_element.text.strip() if price_element else "£0.00" #  Extrae el texto del enlace y quita espacios
            price = float (price_text.lstrip('Â£')) # Elimina los caracteres "Â" y "£" del inicio y convierte a número decimal
            
            # RATING 
            rating_element = book.find ('p', class_= 'star-rating')
            rating_class = rating_element['class'][1] if rating_element else 'Zero' 
            # La clase CSS indica el rating
            rating_map = {'One': 1, 'Two' : 2 , 'Three': 3, 'Four': 4, 'Five': 5, "Zero" : 0 } 
            rating = rating_map.get(rating_class, 0) 
            # Convierte la palabra en inglés a número 
            
            # STOCK 
            stock_element = book.find ('p', class_= 'instock availability')
            in_stock = 'In stock' in stock_element.text if stock_element else False 
            # Verifica si el texto contiene "In stock" → True/False
            
            books.append({ # se agrega todos los datos del libro al diccionario 
                'title': title,
                'price': price,
                'rating' : rating,
                'in_stock' : in_stock,
                'url': book_url
                # NOTA: quantity se obtiene después en paralelo
            })
            
        except Exception as e:
            print(f"❌ Error procesando libro: {e}") 
            continue
    
    # OPTIMIZACIÓN: En vez de obtener la cantidad uno por uno (secuencial),
    # usamos ThreadPoolExecutor para hacer hasta 10 peticiones al mismo tiempo (paralelo)
    # Esto convierte 20 peticiones de 0.5s cada una (10s total) en ~1s total
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Lanza todas las peticiones de cantidad en paralelo
        future_to_book = {
            executor.submit(book_quantity, book['url']): book 
            for book in books
        }
        # A medida que cada petición termina, asigna la cantidad al libro
        for future in as_completed(future_to_book):
            book = future_to_book[future]
            book['quantity'] = future.result()
    
    return books 

def scrape_all_books (): 
    ''' SCRAPE TODOS LOS LIBROS DEL SITIO '''
    all_books = []
    categories = get_all_categories() # obtiene la lista de categorias
    for i, category in enumerate(categories): 
        print (f"Procesando categoria {i+1}/ {len(categories)}: {category['name']}")
        
        page_num = 1 
        current_url = category['url'] # URL de la primera pagina de la categoria 
        
        while current_url: # sigue el bucle mientras haya paginas
            print (f" Pagina {page_num}")
            books_on_page = scrape_books_from_page(current_url) 
            # Extare todos los libros de la pagina actual 
            
            for book in books_on_page:
                book['category'] = category['name']
                # agrega el nombre de la categoria a cada libro 
                
            all_books.extend(books_on_page) # se agrega los libros a la lista total 
            
            # BUSCAR SIGUIENTE PAGINA 
            response = session.get(current_url) # usando session
            soup = BeautifulSoup(response.content, 'html.parser')
            next_button = soup.find('li', class_= 'next')
            # busca el boton next para ir a la siguiente pagina 
            
            if next_button and next_button.find('a'):
                next_url = next_button.find('a')['href']
                current_url = urljoin(current_url, next_url)
                # construye la URL a la siguiente pagina 
                page_num += 1 
            else:
                current_url = None # no hay mas paginas y sale del while 
                
            time.sleep (0.3) # ser amigables con el servidor
        
    print (f" 🎉 Scraping completado: {len(all_books)} libros encontrados")
    return all_books


# BLOQUE 3: EJECUTAR EL SCRAPING 
books_data = scrape_all_books()

# Mostrar muestra de datos 
print ("\n Muestra de los primeros 3 libros: ")
for i, book in enumerate(books_data[:3]):
    print(f"{i+1}. {book['title']} - {book['price']} - ⭐{book['rating']} - {book['category']} - {book['quantity']}")

✅ Librerías importadas correctamente
🕷️ Comenzando la infiltración en Books To Scrape...
🎯 Encontradas 50 categorias
Procesando categoria 1/ 50: Travel
 Pagina 1
Procesando categoria 2/ 50: Mystery
 Pagina 1
 Pagina 2
Procesando categoria 3/ 50: Historical Fiction
 Pagina 1
 Pagina 2
Procesando categoria 4/ 50: Sequential Art
 Pagina 1
 Pagina 2
 Pagina 3
 Pagina 4
Procesando categoria 5/ 50: Classics
 Pagina 1
Procesando categoria 6/ 50: Philosophy
 Pagina 1
Procesando categoria 7/ 50: Romance
 Pagina 1
 Pagina 2
Procesando categoria 8/ 50: Womens Fiction
 Pagina 1
Procesando categoria 9/ 50: Fiction
 Pagina 1
 Pagina 2
 Pagina 3
 Pagina 4
Procesando categoria 10/ 50: Childrens
 Pagina 1
 Pagina 2
Procesando categoria 11/ 50: Religion
 Pagina 1
Procesando categoria 12/ 50: Nonfiction
 Pagina 1
 Pagina 2
 Pagina 3
 Pagina 4
 Pagina 5
 Pagina 6
Procesando categoria 13/ 50: Music
 Pagina 1
Procesando categoria 14/ 50: Default
 Pagina 1
 Pagina 2
 Pagina 3
 Pagina 4
 Pagina 5
 Pagina 6
 P

## Funciones para obtener autores con Google Books API

In [ ]:
def get_author_from_google_books(title, max_retries=3):
    ''' Obtiene la informacion del autor usando Google Books API '''
    
    for attempt in range (max_retries):
        try:
            # Limpiar el título para mejor búsqueda
            clean_title = re.sub(r'[^\w\s]', '', title) 
            url = "https://www.googleapis.com/books/v1/volumes" # url de la base de Google Books API 
            params = {"q": f"intitle:{title}", "maxResults": 1} 
            # q: intitle:{title} — busca libros cuyo título contenga el texto dado.
            # maxResults: 1 — solo pide 1 resultado (el más relevante).
            
            response = requests.get(url, params=params) 
            # hace la petición HTTP GET a la API. requests se encarga de construir la URL final con los parámetros
            
            if response.status_code == 200: # verifica que la respuesta fue exitosa, y es 200 porque es un estandar del protocolo HTTP
                data = response.json() 
                # Convierte la respuesta (que viene en formato JSON) a un diccionario de Python.

                if 'items' in data and len(data['items']) > 0:
                    # Verifica que la API devolvió resultados. La clave 'items' contiene la lista de libros encontrados.
                    
                    volume_info = data['items'][0].get('volumeInfo', {})   
                    #Toma el primer resultado ([0]) y extrae su información del volumen. Si no existe 'volumeInfo', devuelve un diccionario vacío {} para evitar errores.
                    authors = volume_info.get('authors', ['Autor Desconocido']) # extrae la lista de autores 
                    description = volume_info.get('description', 'Sin descripción') # extrae la descripcion del libro 

                    return {
                        'authors': authors,
                        'description': description[:500] + '...' if len(description) > 500 else description
                        # Si la descripción tiene más de 500 caracteres, la recorta y le agrega '...' al final.
                    }
                else:
                    return {
                        'authors': ['Autor Desconocido'],
                        'description': 'Sin descripción' 
                        # si la API no encontro ningun libro, devuelve valores por defecto 
                    }

            else:
                time.sleep(1)  # espera y reintenta 
                continue
            
        except Exception as e:
            print(f"❌ Error consultando API para '{title}': {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
                continue
            # si aun quedan reintentos, se espera 2 segundos y vuelve a intentar 
            else:
                
                return {
                    'authors': ['Autor Desconocido'],
                    'description': 'Sin descripción'
            # si se agotaron devuelve los valores por defecto 
                }
    
    # Si llegamos aquí, todos los intentos fallaron
    return {
        'authors': ['Autor Desconocido'],
        'description': 'Sin descripción'
    }

def enrich_books_with_authors(books_list): # recibe la lista completa que se obtuvo del scraping
    """Enriquece la lista de libros con información de autores"""
    print("🔍 Consultando Google Books API para obtener autores...")

    enriched_books = []
    total_books = len(books_list)
    # Crea una lista vacía para los libros enriquecidos y guarda el total de libros para mostrar progreso.

    # OPTIMIZACIÓN: En vez de consultar la API uno por uno (secuencial),
    # usamos ThreadPoolExecutor para hacer hasta 10 peticiones al mismo tiempo (paralelo)
    completed = 0
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Lanza todas las consultas a la API en paralelo
        future_to_book = {
            executor.submit(get_author_from_google_books, book['title']): book
            for book in books_list
        }
        # A medida que cada petición termina, enriquece el libro con la info del autor
        for future in as_completed(future_to_book):
            book = future_to_book[future]
            author_info = future.result()

            enriched_book = book.copy() # se crea para no modificar la original
            enriched_book.update(author_info) # Fusiona la info del autor (authors y description) dentro del diccionario del libro
            enriched_books.append(enriched_book) # y se agrega el libro enriquecido a la lista final

            completed += 1
            # Mostrar progreso cada 10 libros
            if completed % 10 == 0:
                print(f"  ✅ Progreso: {completed}/{total_books} libros procesados")

    print("🎊 Enriquecimiento completado!")
    return enriched_books




# BLOQUE 5: Ejecutar enriquecimiento con API
print("🌟 Enriqueciendo datos con información de autores...")
enriched_books = enrich_books_with_authors(books_data) # se le pasa los libros del scraping 

# Mostrar muestra de datos enriquecidos
print("\n📊 Muestra de libros enriquecidos:")
for i, book in enumerate(enriched_books[:3]):
    authors_str = ', '.join(book['authors']) 
    print(f"{i+1}. '{book['title']}' por {authors_str}")
    print(f"   Precio: {book['price']} | Rating: ⭐{book['rating']} | Categoría: {book['category']}")
    # Resumen de cada libro 
    print()

🌟 Enriqueciendo datos con información de autores...
🔍 Consultando Google Books API para obtener autores...
  ✅ Progreso: 10/1000 libros procesados
  ✅ Progreso: 20/1000 libros procesados
  ✅ Progreso: 30/1000 libros procesados
  ✅ Progreso: 40/1000 libros procesados
  ✅ Progreso: 50/1000 libros procesados
  ✅ Progreso: 60/1000 libros procesados
  ✅ Progreso: 70/1000 libros procesados
  ✅ Progreso: 80/1000 libros procesados
  ✅ Progreso: 90/1000 libros procesados
  ✅ Progreso: 100/1000 libros procesados
  ✅ Progreso: 110/1000 libros procesados
  ✅ Progreso: 120/1000 libros procesados
  ✅ Progreso: 130/1000 libros procesados
  ✅ Progreso: 140/1000 libros procesados
  ✅ Progreso: 150/1000 libros procesados
  ✅ Progreso: 160/1000 libros procesados
  ✅ Progreso: 170/1000 libros procesados
  ✅ Progreso: 180/1000 libros procesados
  ✅ Progreso: 190/1000 libros procesados
  ✅ Progreso: 200/1000 libros procesados
  ✅ Progreso: 210/1000 libros procesados
  ✅ Progreso: 220/1000 libros procesados


In [11]:

# BLOQUE 6: Diseño UML y Estructura de Base de Datos


print("""
🏗️ DIAGRAMA UML - Estructura de la Base de Datos

┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│     AUTORES     │    │  LIBROS_AUTORES  │    │     LIBROS      │
├─────────────────┤    │   (Tabla Pivot)  │    ├─────────────────┤
│ id (PK)         │◄───┤ autor_id (FK)    │───►│ id (PK)         │
│ nombre          │    │ libro_id (FK)    │    │ titulo          │
│                 │    └──────────────────┘    │ precio          │
└─────────────────┘                           │ rating          │
                                              │ categoria       │
┌─────────────────┐                           │ en_stock        │
|                 |                           | cantidad        |
│   CATEGORIAS    │                           │                 │
├─────────────────┤                           │ descripcion     │
│ id (PK)         │◄──────────────────────────│ categoria_id (FK)│
│ nombre          │                           │ url             │
│                 │                           └─────────────────┘
└─────────────────┘

Relaciones:
- AUTORES ↔ LIBROS: Muchos a Muchos (un autor puede escribir varios libros, 
  un libro puede tener varios autores)
- CATEGORIAS ↔ LIBROS: Uno a Muchos (una categoría tiene muchos libros)
""")


🏗️ DIAGRAMA UML - Estructura de la Base de Datos

┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│     AUTORES     │    │  LIBROS_AUTORES  │    │     LIBROS      │
├─────────────────┤    │   (Tabla Pivot)  │    ├─────────────────┤
│ id (PK)         │◄───┤ autor_id (FK)    │───►│ id (PK)         │
│ nombre          │    │ libro_id (FK)    │    │ titulo          │
│                 │    └──────────────────┘    │ precio          │
└─────────────────┘                           │ rating          │
                                              │ categoria       │
┌─────────────────┐                           │ en_stock        │
|                 |                           | cantidad        |
│   CATEGORIAS    │                           │                 │
├─────────────────┤                           │ descripcion     │
│ id (PK)         │◄──────────────────────────│ categoria_id (FK)│
│ nombre          │                           │ url             │
│                 

In [ ]:

# BLOQUE 7: Creación de Base de Datos (DDL)

def create_database():
    """Crea la base de datos y todas las tablas"""
    
    conn = sqlite3.connect('books_detective.db') # asignamos el nombre a la base de datos
    cursor = conn.cursor()
    
    # DDL - Data Definition Language
    print("🏗️ Creando estructura de base de datos...")
    
    # Tabla Categorías
    cursor.execute(""" 
    CREATE TABLE IF NOT EXISTS categorias (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nom bre VARCHAR(100) UNIQUE NOT NULL
    )
    """)
    # Texto corto, hasta 100 chars
    # Texto medio, hasta 200 chars
    # Texto largo, hasta 500 chars
    # Texto muy largo, hasta 1000 chars
    # TEXT = Texto sin límite 
    # 
    
    # Tabla Autores
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS autores (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre VARCHAR(200) UNIQUE NOT NULL
    )
    """)
    # UNIQUE NOT NULL = no puede estar vacio ni repetirse 
    
    
    # Tabla Libros
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS libros (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        titulo VARCHAR(500) NOT NULL,
        precio DECIMAL(10,2),
        rating INTEGER CHECK(rating >= 0 AND rating <= 5),
        categoria_id INTEGER,
        en_stock BOOLEAN,
        cantidad INTEGER,
        descripcion TEXT,
        url VARCHAR(1000),
        FOREIGN KEY (categoria_id) REFERENCES categorias(id)
    )
    """)
    # CADA LINEA ES UNA COLUMNA EN LA BASE DE DATOS 
    
    
    
    # Tabla de relación muchos a muchos
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS libros_autores (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        libro_id INTEGER,
        autor_id INTEGER,
        FOREIGN KEY (libro_id) REFERENCES libros(id) ON DELETE CASCADE, 
        FOREIGN KEY (autor_id) REFERENCES autores(id) ON DELETE CASCADE,
        UNIQUE(libro_id, autor_id)
    )
    """)   
    conn.commit() # es para poder realizar todos los cambios  
    print("✅ Estructura de base de datos creada exitosamente")
    
    return conn

# Crear la base de datos
conn = create_database()

# BLOQUE 8: Inserción de Datos (DML)


def insert_data(enriched_books, connection):
    """Elimina datos existentes e inserta nuevos datos en la base de datos"""
    
    cursor = connection.cursor()
    print("🧹 Eliminando contenido existente de las tablas...")
    
    # Eliminar contenido de las tablas en el orden correcto para evitar conflictos de claves foráneas
    cursor.execute("DELETE FROM libros_autores")
    cursor.execute("DELETE FROM libros")
    cursor.execute("DELETE FROM autores")
    cursor.execute("DELETE FROM categorias")
    
    # Reiniciar los IDs autoincrementales
    cursor.execute("DELETE FROM sqlite_sequence WHERE name='libros_autores'")
    cursor.execute("DELETE FROM sqlite_sequence WHERE name='libros'")
    cursor.execute("DELETE FROM sqlite_sequence WHERE name='autores'")
    cursor.execute("DELETE FROM sqlite_sequence WHERE name='categorias'")
    # si no hago esto tengo archivos duplicados 
    
    
    connection.commit()
    print("✅ Tablas limpiadas y IDs reiniciados exitosamente")
    
    print("📝 Insertando datos en la base de datos...")
    
    # DML - Data Manipulation Language
    
    # 1. Insertar categorías únicas
    categories = set(book['category'] for book in enriched_books) # descarta categorias duplicadas 
    for category in categories:
        cursor.execute("""
            INSERT OR IGNORE INTO categorias (nombre) VALUES (?)
        """, (category,))
    
    # 2. Insertar autores únicos
    all_authors = set() # no permite duplicados 
    for book in enriched_books:
        for author in book['authors']:
            all_authors.add(author)
    
    for author in all_authors:
        cursor.execute("""
            INSERT OR IGNORE INTO autores (nombre) VALUES (?)
        """, (author,))
    
    # 3. Insertar libros
    for book in enriched_books:
        # Obtener category_id
        cursor.execute("SELECT id FROM categorias WHERE nombre = ?", (book['category'],))
        categoria_id = cursor.fetchone()[0] # busca unicamente uno y luego le paso el indice que necesito 
        
        cursor.execute("""
            INSERT INTO libros (
                titulo, precio, rating, categoria_id, en_stock, cantidad,
                descripcion, url
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?) 
        """, (       # cada marcado es un placeholder ( marcado de posicion )
                # Inyeccion SQL  es una vulnerabilidad de seguridad donde un atacante 
                # inserta código SQL malicioso en las entradas de una aplicación para manipular la base de datos.
                # Se pone el VALUES para prevenir inyecciones SQL 

            book['title'],
            book['price'],
            book['rating'],
            categoria_id,
            book['in_stock'],
            book['quantity'],
            book['description'],
            book['url']
        ))
        
        libro_id = cursor.lastrowid 
        # devuelve el ID que SQLite asignó automáticamente a la última fila insertada con AUTOINCREMENT.
        
        # 4. Insertar relaciones libro-autor
        for author in book['authors']:
            cursor.execute("SELECT id FROM autores WHERE nombre = ?", (author,))
            autor_id = cursor.fetchone()[0]
            
            cursor.execute("""
                INSERT OR IGNORE INTO libros_autores (libro_id, autor_id) 
                VALUES (?, ?)
            """, (libro_id, autor_id))
    
    connection.commit()
    print("✅ Datos insertados exitosamente")
    
    # Mostrar estadísticas
    cursor.execute("SELECT COUNT(*) FROM libros")
    total_books = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM autores")
    total_authors = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM categorias")
    total_categories = cursor.fetchone()[0]
    
    print(f"📊 Estadísticas:")
    print(f"   📚 Libros: {total_books}")
    print(f"   ✍️ Autores: {total_authors}")
    print(f"   🏷️ Categorías: {total_categories}")

# Insertar todos los datos
insert_data(enriched_books, conn)

🏗️ Creando estructura de base de datos...
✅ Estructura de base de datos creada exitosamente
🧹 Eliminando contenido existente de las tablas...
✅ Tablas limpiadas y IDs reiniciados exitosamente
📝 Insertando datos en la base de datos...
✅ Datos insertados exitosamente
📊 Estadísticas:
   📚 Libros: 1000
   ✍️ Autores: 811
   🏷️ Categorías: 50


In [ ]:

# BLOQUE 9: Consultas SQL🕵️‍♂️


def execute_query(connection, query, description):
    """Ejecuta una consulta y muestra los resultados"""
    print(f"\n🔍 {description}")
    print("=" * 60)
    
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall() # busca todas las filas    
    columns = [desc[0] for desc in cursor.description]
    
    # Crear DataFrame para mejor visualización
    df = pd.DataFrame(results, columns=columns)
    print(df.to_string(index=False))
    
    return df

# CONSULTA 1: Libros baratos pero bien rankeados (el santo grial del lector pobre)
query1 = """
SELECT 
    l.titulo,
    l.precio,
    l.rating,
    c.nombre as categoria,
    GROUP_CONCAT(a.nombre, ', ') as autores
FROM libros l
JOIN categorias c ON l.categoria_id = c.id
JOIN libros_autores la ON l.id = la.libro_id
JOIN autores a ON la.autor_id = a.id
WHERE l.rating >= 4 AND l.precio < 20
GROUP BY l.id, l.titulo, l.precio, l.rating, c.nombre
ORDER BY l.rating DESC, l.precio ASC
LIMIT 10
"""

execute_query(conn, query1, 
            "💰 CONSULTA 1: Libros baratos pero bien rankeados (para lectores con presupuesto ajustado)")

# CONSULTA 2: Los autores más prolíficos (¿quién no para de escribir?)
query2 = """
SELECT 
    a.nombre as autor,
    COUNT(l.id) as total_libros,
    AVG(l.rating) as rating_promedio,
    AVG(l.precio) as precio_promedio,
    MIN(l.precio) as libro_mas_barato,
    MAX(l.precio) as libro_mas_caro
FROM autores a
JOIN libros_autores la ON a.id = la.autor_id
JOIN libros l ON la.libro_id = l.id
GROUP BY a.id, a.nombre
HAVING COUNT(l.id) > 1
ORDER BY total_libros DESC, rating_promedio DESC
LIMIT 15
"""

execute_query(conn, query2, 
            "✍️ CONSULTA 2: Autores más prolíficos (los que no paran de escribir)")

# CONSULTA 3: Análisis por categorías (¿dónde está el oro literario?)
query3 = """
SELECT 
    c.nombre as categoria,
    COUNT(l.id) as total_libros,
    AVG(l.rating) as rating_promedio, 
    AVG(l.precio) as precio_promedio,
    COUNT(CASE WHEN l.rating >= 4 THEN 1 END) as libros_4_o_mas_estrellas,
    ROUND(COUNT(CASE WHEN l.rating >= 4 THEN 1 END) * 100.0 / COUNT(l.id), 2) as porcentaje_buenos_libros
FROM categorias c
JOIN libros l ON c.id = l.categoria_id
GROUP BY c.id, c.nombre
ORDER BY rating_promedio DESC, porcentaje_buenos_libros DESC
"""

execute_query(conn, query3, 
            "📊 CONSULTA 3: Análisis por categorías (¿cuál género tiene los mejores libros?)")



# BLOQUE 10: Consultas Complejas con Window Functions y Subconsultasv


# CONSULTA 4: Ranking de libros por categoría usando Window Functions
query4 = """
SELECT 
    titulo,
    precio,
    rating,
    categoria,
    autores,
    ranking_en_categoria,
    total_en_categoria
FROM (
    SELECT 
        l.titulo,
        l.precio,
        l.rating,
        c.nombre as categoria,
        GROUP_CONCAT(a.nombre, ', ') as autores,
        ROW_NUMBER() OVER (
            PARTITION BY c.nombre 
            ORDER BY l.rating DESC, l.precio ASC
        ) as ranking_en_categoria,
        COUNT(*) OVER (PARTITION BY c.nombre) as total_en_categoria
    FROM libros l
    JOIN categorias c ON l.categoria_id = c.id
    JOIN libros_autores la ON l.id = la.libro_id
    JOIN autores a ON la.autor_id = a.id
    GROUP BY l.id, l.titulo, l.precio, l.rating, c.nombre
) ranked
WHERE ranking_en_categoria <= 3
ORDER BY categoria, ranking_en_categoria
"""
# el JOIN ahi sirve para calcular stats de autores y filtrar antes de unir


execute_query(conn, query4, 
            "🏆 CONSULTA 4: Top 3 libros por categoría (usando Window Functions)")

# CONSULTA 5: Los peores libros de los mejores autores (subconsulta compleja)
query5 = """
SELECT 
    l.titulo,
    l.rating,
    l.precio,
    a.nombre as autor,
    autor_stats.rating_promedio_autor,
    autor_stats.total_libros_autor
FROM libros l
JOIN libros_autores la ON l.id = la.libro_id
JOIN autores a ON la.autor_id = a.id
JOIN ( 
    SELECT 
        a2.id,
        AVG(l2.rating) as rating_promedio_autor,
        COUNT(l2.id) as total_libros_autor
    FROM autores a2
    JOIN libros_autores la2 ON a2.id = la2.autor_id
    JOIN libros l2 ON la2.libro_id = l2.id
    GROUP BY a2.id
    HAVING AVG(l2.rating) >= 3.5 AND COUNT(l2.id) >= 2
) autor_stats ON a.id = autor_stats.id
WHERE l.rating <= 2
ORDER BY autor_stats.rating_promedio_autor DESC, l.rating ASC
"""

execute_query(conn, query5, 
            "😱 CONSULTA 5: Los peores libros de los mejores autores (¿hasta los buenos fallan?)")



🔍 💰 CONSULTA 1: Libros baratos pero bien rankeados (para lectores con presupuesto ajustado)
                                                                                 titulo  precio  rating      categoria                     autores
                                                             An Abundance of Katherines   10.00       5    Young Adult                  John Green
                                                                   Greek Mythic History   10.23       5        Default       History Brought Alive
                             The Power Greens Cookbook: 140 Delicious Superfood Recipes   11.05       5 Food and Drink                 Dana Jacobi
                                                                     Dear Mr. Knightley   11.21       5        Fiction              Katherine Reay
                                                                    The Darkest Corners   11.33       5    Young Adult                 Kara Thomas
Naturally Lean: 125 Nouri

,titulo,rating,precio,autor,rating_promedio_autor,total_libros_autor
0,Barefoot Contessa Back to Basics,1,28.01,Ina Garten,3.666667,3
1,The Raven King (The Raven Cycle #4),2,30.57,Maggie Stiefvater,3.666667,3
2,City of Bones (The Mortal Instruments #1),1,43.28,Cassandra Clare,3.500000,4
3,The Da Vinci Code (Robert Langdon #2),2,22.96,Dan Brown,3.500000,4
4,The Hidden Oracle (The Trials of Apollo #1),2,52.26,Rick Riordan,3.500000,2
5,Of Mice and Men,2,47.11,John Steinbeck,3.500000,2
6,The Power of Now: A Guide to Spiritual Enlight...,2,43.54,Eckhart Tolle,3.500000,2


In [ ]:

# BLOQUE 11: Demostración de Indexación - ANTES y DESPUÉS


print("\n⚡ DEMOSTRACIÓN DE INDEXACIÓN - El Poder de la Optimización")
print("=" * 70)

# Consulta que será lenta sin índice
slow_query = """
SELECT 
    l.titulo,
    l.precio,
    l.rating,
    a.nombre as autor
FROM libros l
JOIN libros_autores la ON l.id = la.libro_id
JOIN autores a ON la.autor_id = a.id
WHERE l.precio BETWEEN 10 AND 30 
AND a.nombre LIKE '%John%'
ORDER BY l.precio DESC
"""
# sin indices recorreria las 1000 filas de libros
# pero con indices solo hace unos pocos saltos en cada arbol 

# ANTES - Sin índice
print("🐌 ANTES: Sin índices optimizados")
cursor = conn.cursor()

# Activar query plan para ver la explicación
cursor.execute("EXPLAIN QUERY PLAN " + slow_query)
plan_before = cursor.fetchall()
print("Plan de ejecución SIN índices:")
for step in plan_before:
    print(f"  {step}")

# Medir tiempo de ejecución
import time
start_time = time.time()
cursor.execute(slow_query)
results_before = cursor.fetchall()
end_time = time.time()
time_before = (end_time - start_time) * 1000  # En milisegundos

print(f"⏱️ Tiempo de ejecución SIN índices: {time_before:.2f} ms")
print(f"📊 Resultados encontrados: {len(results_before)}")

# DESPUÉS - Con índices optimizados
print("\n🚀 DESPUÉS: Con índices optimizados")

# Crear índices estratégicos
cursor.execute("CREATE INDEX IF NOT EXISTS idx_libros_precio ON libros(precio)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_autores_nombre ON autores(nombre)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_libros_autores_libro_id ON libros_autores(libro_id)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_libros_autores_autor_id ON libros_autores(autor_id)")

# Ver el nuevo plan de ejecución
cursor.execute("EXPLAIN QUERY PLAN " + slow_query)
plan_after = cursor.fetchall()
print("Plan de ejecución CON índices:")
for step in plan_after:
    print(f"  {step}")

# Medir tiempo de ejecución con índices
start_time = time.time()
cursor.execute(slow_query)
results_after = cursor.fetchall()
end_time = time.time()
time_after = (end_time - start_time) * 1000  # En milisegundos

print(f"⏱️ Tiempo de ejecución CON índices: {time_after:.2f} ms")
print(f"📊 Resultados encontrados: {len(results_after)}")

# Calcular mejora
if time_before > 0:
    improvement = ((time_before - time_after) / time_before) * 100
    print(f"🎯 Mejora de rendimiento: {improvement:.1f}%")
    print(f"⚡ Speedup: {time_before/time_after if time_after > 0 else 'N/A'}x más rápido")



⚡ DEMOSTRACIÓN DE INDEXACIÓN - El Poder de la Optimización
🐌 ANTES: Sin índices optimizados
Plan de ejecución SIN índices:
  (6, 0, 0, 'SEARCH l USING INDEX idx_libros_precio (precio>? AND precio<?)')
  (12, 0, 0, 'SEARCH la USING COVERING INDEX sqlite_autoindex_libros_autores_1 (libro_id=?)')
  (16, 0, 0, 'SEARCH a USING INTEGER PRIMARY KEY (rowid=?)')
⏱️ Tiempo de ejecución SIN índices: 17.00 ms
📊 Resultados encontrados: 8

🚀 DESPUÉS: Con índices optimizados
Plan de ejecución CON índices:
  (6, 0, 0, 'SEARCH l USING INDEX idx_libros_precio (precio>? AND precio<?)')
  (12, 0, 0, 'SEARCH la USING COVERING INDEX sqlite_autoindex_libros_autores_1 (libro_id=?)')
  (16, 0, 0, 'SEARCH a USING INTEGER PRIMARY KEY (rowid=?)')
⏱️ Tiempo de ejecución CON índices: 5.00 ms
📊 Resultados encontrados: 8
🎯 Mejora de rendimiento: 70.6%
⚡ Speedup: 3.402605459057072x más rápido


In [ ]:

# BLOQUE 12: Mini Reporte Final del Detective


def generate_detective_report(connection):
    """Genera el reporte final del detective de datos"""
    
    print("\n" + "="*80)
    print("🕵️‍♂️ REPORTE FINAL DEL DETECTIVE DE DATOS PEOR PAGADO DEL HEMISFERIO SUR")
    print("="*80)
    
    cursor = connection.cursor()
    
    # Estadísticas generales
    cursor.execute("SELECT COUNT(*) FROM libros")
    total_books = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM autores")
    total_authors = cursor.fetchone()[0]
    
    cursor.execute("SELECT AVG(precio) FROM libros")
    avg_price = cursor.fetchone()[0]
    
    cursor.execute("SELECT AVG(rating) FROM libros")
    avg_rating = cursor.fetchone()[0]
    
    print(f"📊 ESTADÍSTICAS GENERALES:")
    print(f"   📚 Total de libros analizados: {total_books}")
    print(f"   ✍️ Total de autores descubiertos: {total_authors}")
    print(f"   💰 Precio promedio: £{avg_price:.2f}")
    print(f"   ⭐ Rating promedio: {avg_rating:.1f}/5")
    
    # El autor más prolífico
    cursor.execute("""
        SELECT a.nombre, COUNT(l.id) as total
        FROM autores a
        JOIN libros_autores la ON a.id = la.autor_id
        JOIN libros l ON la.libro_id = l.id
        GROUP BY a.nombre
        ORDER BY total DESC
        LIMIT 1
    """)
    top_author = cursor.fetchone()
    print(f"\n👑 AUTOR MÁS PROLÍFICO: {top_author[0]} con {top_author[1]} libros")
    
    # La categoría más cara
    cursor.execute("""
        SELECT c.nombre, AVG(l.precio) as precio_prom
        FROM categorias c
        JOIN libros l ON c.id = l.categoria_id
        GROUP BY c.nombre
        ORDER BY precio_prom DESC
        LIMIT 1
    """)
    expensive_cat = cursor.fetchone()
    print(f"💸 CATEGORÍA MÁS CARA: {expensive_cat[0]} (£{expensive_cat[1]:.2f} promedio)")
    
    # El libro más caro
    cursor.execute("""
        SELECT l.titulo, l.precio, GROUP_CONCAT(a.nombre, ', ') as autores
        FROM libros l
        JOIN libros_autores la ON l.id = la.libro_id
        JOIN autores a ON la.autor_id = a.id
        GROUP BY l.id
        ORDER BY l.precio DESC
        LIMIT 1
    """)
    expensive_book = cursor.fetchone()
    print(f"💎 LIBRO MÁS CARO: '{expensive_book[0]}' por {expensive_book[2]} - £{expensive_book[1]}")
    
    # Libros de 1 estrella (las catástrofes)
    cursor.execute("""
        SELECT COUNT(*) FROM libros WHERE rating = 1
    """)
    disasters = cursor.fetchone()[0]
    percentage = (disasters / total_books) * 100
    print(f"💥 CATÁSTROFES LITERARIAS: {disasters} libros con 1 estrella ({percentage:.1f}%)")
    
    print("\n🎭 CONCLUSIONES DEL DETECTIVE:")
    print("   • El crimen literario está resuelto")
    print("   • Los memes han sido documentados")
    print("   • El pingüino jefe estaría orgulloso")
    print("   • Mission accomplished! 🎯")
    
    print("\n" + "="*80)
    print("FIN DEL REPORTE - Caso cerrado 📋✅")
    print("="*80)

# Generar reporte final
generate_detective_report(conn)


# BLOQUE 13: Limpieza y cierre

# Guardar una copia de los datos en CSV para análisis adicional
print("\n💾 Guardando datos en CSV para análisis posteriores...")

# Query para obtener datos completos
full_query = """
SELECT 
    l.titulo,
    l.precio,
    l.rating,
    c.nombre as categoria,
    l.en_stock,
    GROUP_CONCAT(a.nombre, ', ') as autores,
    l.descripcion
FROM libros l
JOIN categorias c ON l.categoria_id = c.id
JOIN libros_autores la ON l.id = la.libro_id
JOIN autores a ON la.autor_id = a.id
GROUP BY l.id, l.titulo, l.precio, l.rating, c.nombre, l.en_stock, l.descripcion
ORDER BY l.rating DESC, l.precio ASC
"""

# Ejecutar query y guardar en CSV
df_export = execute_query(conn, full_query, "📁 Exportando datos completos")
df_export.to_csv('books_detective_complete_data.csv', index=False, encoding='utf-8')
print("✅ Datos guardados en 'books_detective_complete_data.csv'")

# Cerrar conexión a la base de datos
conn.close()
print("🔒 Conexión a base de datos cerrada") 

print("\n🎉 ¡CHALLENGE COMPLETADO EXITOSAMENTE!")
print("🏆 Has desbloqueado el achievement: 'Detective de Datos Master'")
print("📚 Ahora eres oficialmente el detective de datos peor pagado... ¡pero el mejor!")

# =============================================================================
# BONUS: Verificación final de que todo está funcionando
# =============================================================================

print("\n🔍 VERIFICACIÓN FINAL:")
print("✅ Web scraping del sitio completo")
print("✅ Integración con Google Books API")
print("✅ Base de datos relacional con relaciones muchos a muchos")
print("✅ Diagrama UML documentado")
print("✅ DDL y DML implementados")
print("✅ 5 consultas emocionales con propósito")
print("✅ Window functions y subconsultas complejas")
print("✅ Demostración de indexación antes/después")
print("✅ Todo en un solo Jupyter Notebook")
print("✅ Reporte final del detective")

print("\n🎭 El pingüino jefe dice: 'Trabajo bien hecho, detective!'")
print("🚀 Listo para enfrentar el próximo desafío de programación!");


🕵️‍♂️ REPORTE FINAL DEL DETECTIVE DE DATOS PEOR PAGADO DEL HEMISFERIO SUR
📊 ESTADÍSTICAS GENERALES:
   📚 Total de libros analizados: 1000
   ✍️ Total de autores descubiertos: 811
   💰 Precio promedio: £35.07
   ⭐ Rating promedio: 2.9/5

👑 AUTOR MÁS PROLÍFICO: Autor Desconocido con 170 libros
💸 CATEGORÍA MÁS CARA: Suspense (£58.33 promedio)
💎 LIBRO MÁS CARO: 'The Perfect Play (Play by Play #1)' por John Vornholt - £59.99
💥 CATÁSTROFES LITERARIAS: 226 libros con 1 estrella (22.6%)

🎭 CONCLUSIONES DEL DETECTIVE:
   • El crimen literario está resuelto
   • Los memes han sido documentados
   • El pingüino jefe estaría orgulloso
   • Mission accomplished! 🎯

FIN DEL REPORTE - Caso cerrado 📋✅

💾 Guardando datos en CSV para análisis posteriores...

🔍 📁 Exportando datos completos
                                                                                                                                                                                                      titulo  precio  rat